###Taller Evaluado 02 Y 03
**Objetivo:** Diseñar y ejecutar un pipeline en Databricks que ingeste, limpie, transforme, una y agregue datos relevantes para su industria elegida, persista resultados en Delta + Unity Catalog, y genere una tabla de auditoría con trazabilidad de la ejecución (origen, conteos y marcas de tiempo). 

### 1. Ingesta y perfilado inicial.
- Lectura de 3–4 datasets (p. ej., clientes, transacciones, catálogo/productos, sucursales, etc.).
- Validar esquema, nulos, rangos de fechas, duplicados.

Busqué DataSets en la página [KAGGLE](https://www.kaggle.com/) y obtuve los **datasets del monitoreo de la calidad del aire en Seúl, Corea del Sur**, y proporciona un caso práctico para analizar los niveles de contaminación en diferentes distritos a lo largo del tiempo. El dataset principal, `Measurement_summary.csv`, contiene mediciones horarias de contaminantes clave como SO2, NO2, CO, O3, PM10 y PM2.5, recolectadas desde enero de 2017 hasta diciembre de 2018. Para enriquecer este análisis, se incluyen dos datasets de apoyo: `Measurement_station_info.csv`, que proporciona detalles geográficos y de ubicación de las 25 estaciones de monitoreo, y `Measurement_item_info.csv`, que describe los códigos de los ítems de medición y los umbrales de calidad del aire asociados a cada contaminante. En conjunto, estos archivos permiten un perfilado de datos integral y la ingesta de un modelo de datos relacional.

El enlace del DataSet en mención es: [Air Pollution in Seoul](https://www.kaggle.com/datasets/bappekim/air-pollution-in-seoul).

### Procedimiento

1. Creamos catalogo y schema
2. Definimos rutas de origen
3. Leemos archivos de origen
4. Limpiamos a los diferentes datasets
5. Creamos columnas derivadas
6. Aplicamos joins para generar nuevos datasets
7. Aplicamos agregaciones para nuevos datasets
8. Escribimos en formato delta

In [0]:
catalog = "dmc_taller02y03"
schema_name = "gold_airpollutionseoul"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_name}")

In [0]:
path_mediciones = "/Volumes/dmc_taller02y03/gold_airpollutionseoul/source/csv/Measurement_info.csv"
path_unidades_medida = "/Volumes/dmc_taller02y03/gold_airpollutionseoul/source/csv/Measurement_item_info.csv"
path_estaciones = "/Volumes/dmc_taller02y03/gold_airpollutionseoul/source/csv/Measurement_station_info.csv"

df_mediciones = spark.read.option("header", True).option("inferSchema", True).csv(path_mediciones)
df_unidades_medida = spark.read.option("header", True).option("inferSchema", True).csv(path_unidades_medida)
df_estacioneses = spark.read.option("header", True).option("inferSchema", True).csv(path_estaciones)

### 2. Limpieza y normalización
- Eliminar/Corregir registros inválidos (nulos en claves, precios/cantidades negativas o cero, fechas inválidas, emails mal formados, etc.). 
- Normalizar strings (trim, lower, regex si aplica) y cast de tipos.

In [0]:
from pyspark.sql.functions import lower, col, trim, when, coalesce, lit, round, to_date, year, month, sum, countDistinct, avg
from pyspark.sql.types import DoubleType

In [0]:
df_mediciones_clean = (
    df_mediciones
    .withColumn("email", when(col("email").contains("@"), col("email")))
    .withColumn("email", coalesce(col("email"), lit("-")))
)

In [0]:
df_productos_clean = (
    df_productos
    .withColumn("nombre_producto", trim(lower(col("nombre_producto"))))
)

In [0]:
df_ventas_clean = (
    df_ventas
    .dropna(subset=["cantidad"])
    .filter(col("precio_unitario") > 0)
)

![](/Volumes/dmc/gold_farmacias/source/Diagrama_AdventureWorksLT_01.png)

In [0]:
df_ventas_procesada = (
    df_ventas_clean
    .withColumn("monto_total", round(col("cantidad").cast(DoubleType()) * col("precio_unitario").cast("double"), 2))
    .withColumn("fecha_venta", to_date(col("fecha_venta")))
    .withColumn("anio_venta", year(col("fecha_venta")))
    .withColumn("mes_venta", month(col("fecha_venta")))
)

In [0]:
df_join = (
    df_ventas_procesada.alias("vp")
    .join(
        df_clientes_clean.alias("cc").select("id_cliente", "nombre", "apellido", "email"),
        col("vp.id_cliente") == col("cc.id_cliente"),
        "left"
    )
    .join(
        df_productos_clean.alias("pc").select("id_producto", "nombre_producto", "categoria", "marca", "costo_unitario"),
        col("vp.id_producto") == col("pc.id_producto"),
        "left"
    )
    .join(
        df_sucursales.alias("s").select("id_sucursal", "ciudad", "distrito", "nombre_sucursal"),
        col("vp.id_sucursal") == col("s.id_sucursal"),
        "left"
    )
    .select(
        col("vp.id_venta").alias("id_venta"),
        col("vp.fecha_venta").alias("fecha_venta"),
        col("vp.id_cliente").alias("id_cliente"),
        col("cc.nombre").alias("nombre"),
        col("cc.apellido").alias("apellido"),
        col("cc.email").alias("email"),
        col("vp.id_producto").alias("id_producto"),
        col("pc.nombre_producto").alias("nombre_producto"),
        col("pc.categoria").alias("categoria"),
        col("pc.marca").alias("marca"),
        col("pc.costo_unitario").alias("costo_unitario"),
        col("vp.id_sucursal").alias("id_sucursal"),
        col("s.ciudad").alias("ciudad"),
        col("s.distrito").alias("distrito"),
        col("s.nombre_sucursal").alias("nombre_sucursal"),
        col("vp.cantidad").alias("cantidad"),
        col("vp.precio_unitario").alias("precio_unitario"),
        col("vp.monto_total").alias("monto_total"),
        # (col("vp.cantidad") * col("vp.precio_unitario")).alias("monto_total_2"),
        col("vp.anio_venta").alias("anio_venta"),
        col("vp.mes_venta").alias("mes_venta")        
    )
).dropDuplicates(["id_venta"])

In [0]:
df_categoria_ticket = (
    df_join
    .withColumn("categoria_ticket", 
                when(col("monto_total") >= 200, lit("Alto"))
                .when(col("monto_total") >= 80, lit("Medio"))
                .otherwise(lit("Bajo"))
    )
)

In [0]:
df_ventas_ciudad_anio = (
    df_join
    .groupBy("ciudad", "anio_venta")
    .agg(
        sum("monto_total").alias("venta_total"),
        countDistinct("id_venta").alias("num_ventas")
    )
)

In [0]:
df_ventas_cliente = (
    df_join
    .groupBy("id_cliente")
    .agg(
        avg("monto_total").alias("venta_promedio"),
        countDistinct("id_venta").alias("num_ventas")
    )
)

In [0]:
df_categoria_ventas = (
    df_categoria_ticket
    .groupBy("categoria_ticket")
    .agg(
        sum("monto_total").alias("ventas"),
        countDistinct("id_venta").alias("num_ventas")
    )
)

In [0]:
tbl_detalle = f"{catalog}.{schema_name}.ventas_detalle"
tbl_ciudad = f"{catalog}.{schema_name}.ventas_ciudad_anio"
tbl_promedio = f"{catalog}.{schema_name}.ventas_promedio"
tbl_categoria = f"{catalog}.{schema_name}.ventas_categoria"

In [0]:
df_join.write.format("delta").mode("overwrite").saveAsTable(tbl_detalle)

In [0]:
df_join.write.format("delta").mode("overwrite").partitionBy("anio_venta", "mes_venta").save("/Volumes/dmc/gold_farmacias/source/ventas_detalle/")

In [0]:
df_ventas_ciudad_anio.write.format("delta").mode("overwrite").saveAsTable(tbl_ciudad)

In [0]:
df_ventas_cliente.write.format("delta").mode("overwrite").saveAsTable(tbl_promedio)

In [0]:
df_categoria_ventas.write.format("delta").mode("overwrite").saveAsTable(tbl_categoria)

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_detalle

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_ciudad_anio

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_promedio

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_categoria